# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

# Hipótesis a evaluar

Las preguntas que estamos buscando evaluar son:

- ¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?
- ¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?
- ¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?
- ¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?

## Abre el archivo de datos y mira la información general. 

In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np

# Carga los datos
data = pd.read_csv('data/credit_scoring_eng.csv')

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [2]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
data.shape

(21525, 12)

In [3]:
# vamos a mostrar las primeras filas N
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [4]:
# Obtener información sobre los datos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Una vez viendo los datos, podemos notar los siguientes puntos:
- Hay datos negativos en *days_employed* por lo cual hay que entender porque se pusieron esos negativos, puede ser que sean errores de entrada de datos o tambien puede ser que signifiquen los dias de desempleo de la persona
- En la tabla *education* hay datos en mayusculas y minusculas, no estan unificados
- Hay datos ausentes en *days_employed* y *total_income*

In [5]:
# Veamos cuantos datos faltantes tenemos en total
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
data[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Viendo los datos, parece que nuestras dos columnas con datos faltantes los presentan en las mismas filas. Esto puede tener sentido ya que sin dias trabajados, no hay ingreso que declarar.

Sin embargo, vamos a filtrar los datos por ambas columnas buscando los faltantes para confirmar esta hipotesis.

In [7]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
data_faltante = data[data['days_employed'].isna()]

# Veamos si en ambas columnas faltan los mismos datos en las columnas
data_faltante[data_faltante['total_income'].isna()].isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Conclusión intermedia**

Una vez hecho el filtrado, podemos confirmar que ambas columnas tienen datos faltantes en las mismas filas. Esto como comentabamos puede estar relacionado con la idea de que si la persona no tiene dias trabajados, no puede presentar ingresos; sin embargo puede ser que reciba ingresos de otra forma y no necesariamente tenga dias trabajados (por ejemplo: inversiones)

Vamos a checar la proporción que representa para las columnas los datos faltantes y además vamos a checar si los tipos de empleo pueden ser la rázon de que haya valores vacios en la columna *days_employed*

In [8]:
# Vamos a checar el porcentaje de valores ausentes en general
porcentaje_nulos = lambda x: x * 100 / data.children.size


data.isna().sum().apply(porcentaje_nulos)

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

In [9]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
data_faltante.income_type.unique()

array(['retiree', 'civil servant', 'business', 'employee', 'entrepreneur'],
      dtype=object)

In [10]:
# Checamos la distribución
porcentaje_nulos = lambda x: x * 100 / data_faltante.children.size
data_faltante.income_type.value_counts().apply(porcentaje_nulos)

employee         50.827967
business         23.367065
retiree          18.997240
civil servant     6.761730
entrepreneur      0.045998
Name: income_type, dtype: float64

Haciendo el analisis de los datos faltantes, pudimos notar que el 50% de los datos faltantes estan empleados, el 23% tiene un negocio, el 19% esta retirado, 6% es funcionario y el resto es emprendedor. 

Que el 50% de nuestros datos nulos sean empleados nos hace confirmar que nuestra hipotesis era incorrecta y que la razón de estos datos nulos sea alguna otra.

**Posibles razones por las que hay valores ausentes en los datos**

Tomando en cuenta el análisis que realizamos, podemos pensar que los datos son ausentes al azar; sin embargo que ambas columnas tengan las mismas filas vacias parece mas deliberado que al azar.

Despejemos primero la idea de que sean valores aleatorios


In [11]:
# Comprobando la distribución en el conjunto de datos entero
porcentaje_nulos = lambda x: x * 100 / data.children.size
data.income_type.value_counts().apply(porcentaje_nulos)

employee                       51.656214
business                       23.623693
retiree                        17.914053
civil servant                   6.778165
unemployed                      0.009292
entrepreneur                    0.009292
student                         0.004646
paternity / maternity leave     0.004646
Name: income_type, dtype: float64

**Conclusión intermedia**

Checando como esta distribuido la información del dataset completo por tipo de empleo, pudimos notar que las distribuciones son casi similares, esto podria indicar que realmente los datos faltantes son accidentales ya que estan en una misma proporción.

**Conclusiones**

Tomando en cuenta las similitudes que hubo en la distribución del dataset completo y el dataset filtrado, consideramos que los datos faltantes son accidentales, representan el 10% de los datos de sus columnas por lo que consideramos que no afectarian de manera significativa al análisis final de estos datos.

Tomando en cuenta que la columna *total_income* es parte del análisis final, consideramos que la mejor opción es eliminar estos datos ya que pueden influir de cierta manera en nuestro análisis y al ser el 10% de ambas columnas, aun tenemos bastante información para proseguir.

En el siguiente apartado, nos enfocaremos en eliminar estos datos ausentes, checar duplicados y tratarlos, unificar nombres de categorias y entender porque hay datos negativos en *days_employed*

## Transformación de datos

Vamos a checar cada columna individualmente para ver si detectamos alguna incosistencia que se haya pasado por alto

Comenzemos checando la columna `education` para poder unificar las categorias

In [12]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
data.education.unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [13]:
# Arregla los registros si es necesario
data.education = data.education.str.lower()

In [14]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
data.education.unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Vamos a pasar a la columna `children` y ver su distribución

In [15]:
# Veamos la distribución de los valores en la columna `children`
data.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Al ver la distribución descubrimos que hay valores extraños entre los datos. Segun los datos hay 76 personas con 20 hijos y 47 con -1 hijos. El primer caso puede darse pero tambien puede deberse a un error de tipificación al querer poner 2 se agrego un 0 al final, esto ultimo puedo repetirse con nuestro segundo caso, se quiso poner 1 y se agrego un - al inicio.

Vamos a ver el porcentaje que representan estos datos problematicos para con base en ello, tomar una decisión

In [16]:
# Checamos el porcentaje
porcentaje_children = lambda x: x * 100 / data.children.size
data.children.value_counts().apply(porcentaje_children)

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64

Representan el 0.56% de nuestros datos por lo que podriamos eliminarlos y no afectarian de manera significativa a nuestro análisis

In [17]:
# Eliminamos los datos problematicos
data.drop(data[data['children'] == 20].index, inplace=True)
data.drop(data[data['children'] == -1].index, inplace=True)

In [18]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
data.children.value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Pasamos a la columna `days_employed`. Aqui tenemos la situación de los datos nulos pero los trataremos más adelante, aqui nos vamos a enfocar en los datos negativos que se presentan en la columna. Vamos a ver el porcentaje que representan a total columna.

Ya que hay bastantes datos unicos, crearemos otra columna que nos diga si es menor o mayor a 0 y poder clasificar mejor los datos

In [19]:
# Creamos la columna days_employed_group para poder ver la distribución
def days_employed_group(days):
    if days < 0:
        return 'Negativos'
    if days >= 0:
        return 'Positivos'
        
data['days_employed_group'] = data['days_employed'].apply(days_employed_group)

# Checamos la distribucion del grupo
porcentaje = lambda x: x * 100 / data.children.size
data.days_employed_group.value_counts().apply(porcentaje)

Negativos    73.866928
Positivos    16.031212
Name: days_employed_group, dtype: float64

Al realizar la clasificación pudimos notar que la cantidad de datos problemáticos es abismal, un 73.89% de todos nuestros datos. Debido a esta proporción no podemos eliminar esos datos.

Tambien nuestra hipotesis temprana que podrian significar los dias de desempleo queda deshechada ya que las personas desempleadas no representan ni un 1% de los datos.

Esta proporción de datos problematicos puede indicar problemas técnicos al pasar los datos y que se colocara un `-` al principio de los datos. Para solucionar este tema, se decidio pasar los datos negativos a positivos y asi poder ocuparlos correctamente

In [20]:
# Pasamos los datos a positivos
cambiar_positivo = lambda x: (x * -1) if x < 0 else x * 1
data.days_employed = data.days_employed.apply(cambiar_positivo)

In [21]:
# Comprueba el resultado - asegúrate de que esté arreglado
data[data.days_employed < 0].count()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
days_employed_group    0
dtype: int64

Otra comprobación que debemos hacer es que nadie de los candidatos pase en dias de experiencia a su `edad - 16 años` (hay que hacer esta resta ya que la edad minima legal para trabajar es de 16 años).

In [22]:
# Primero creemos una columna pasando los dias de experiencia a años de experiencia
get_years_experience = lambda x: x / 365
data['years_employed'] = data.days_employed.apply(get_years_experience)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_group,years_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Negativos,23.116912
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Negativos,11.026860
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,Negativos,15.406637
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Negativos,11.300677
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Positivos,932.235814


In [23]:
# Veamos cuantos datos no cumplen con nuestra regla
print(data[data.years_employed > (data.dob_years - 16)]['days_employed'].count())
porcentaje_overlimit_days_employed = data[data.years_employed > (data.dob_years -16)]['days_employed'].count() / data.children.count()
print(f'{porcentaje_overlimit_days_employed:.2%}')

3593
16.79%


El 16.79% de los datos son mayores a los años de vida de las personas, lo cual no puede ser posible y nos indica un error en el ingreso de la información. Ya que son datos decimales podria ser que el error se deba a que los puntos se desplazaron más decimas de lo que deberian.

Verifiquemos cual es el número máximo de dias de experiencia que son problemáticos y en base a ello tomemos una decisión.

In [24]:
print(data[data.years_employed > (data.dob_years - 16)]['days_employed'].max())

401755.40047533


Viendo el número máximo, se decide que se recorrera el punto decimal 2 cifras a la izquierda

In [25]:
# Aplicamos la operación
data.days_employed = data[data.years_employed > (data.dob_years - 16)]['days_employed'] / 100
data['years_employed'] = data.days_employed.apply(get_years_experience)

In [26]:
# Verificamos los ajustes
print(data[data.years_employed > (data.dob_years - 16)]['days_employed'].count())
porcentaje_overlimit_days_employed = data[data.years_employed > (data.dob_years -16)]['days_employed'].count() / data.children.count()
print(f'{porcentaje_overlimit_days_employed:.2%}')

92
0.43%


Una vez hecho el ajuste, aun tenemos 92 datos problematicos, sin embargo al representar el 0.43% de nuestra data, podemos eliminarlos sin ningun problema.

In [27]:
# Se eliminan datos problematicos restantes
data.drop(data[data.years_employed > (data.dob_years - 16)].index, inplace=True)
# Verificamos la operación
print(data[data.years_employed > (data.dob_years - 16)]['days_employed'].count())
porcentaje_overlimit_days_employed = data[data.years_employed > (data.dob_years -16)]['days_employed'].count() / data.children.count()
print(f'{porcentaje_overlimit_days_employed:.2%}')

0
0.00%


Pasemos a la columna `dob_years`. Veamos su distribución a ver si detectamos algun dato problematico

In [28]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
data.dob_years.value_counts().apply(porcentaje)

35    2.881276
41    2.829657
40    2.829657
34    2.801502
38    2.792116
42    2.778038
33    2.707649
39    2.684186
31    2.609104
36    2.595026
44    2.548099
29    2.548099
30    2.515251
48    2.515251
37    2.491788
43    2.393243
50    2.388550
32    2.374472
49    2.369779
28    2.351009
45    2.318160
27    2.299390
52    2.266542
56    2.261849
47    2.252464
54    2.233693
46    2.200845
58    2.163304
57    2.144533
53    2.144533
51    2.092914
59    2.069451
55    2.069451
26    1.900516
60    1.764430
25    1.670577
61    1.656499
62    1.647114
63    1.257626
64    1.234162
24    1.234162
23    1.182543
65    0.910371
66    0.858752
22    0.854059
67    0.783670
21    0.516190
68    0.464571
69    0.389489
70    0.305021
71    0.272173
20    0.239324
72    0.154857
19    0.065697
0     0.046926
73    0.037541
74    0.028156
75    0.004693
Name: dob_years, dtype: float64

Al checar la distribución de `dob_years` pudimos notar que hay datos que fueron llenados con 0, ninguna persona con 0 años podria estar en los datos, seguramente algun error de dedo al colocar los datos. Ya que representan el 0.46% de los datos, vamos a eliminarlos

In [29]:
# Resuelve los problemas en la columna `dob_years`, si existen
data.drop(data[data['dob_years'] == 0].index, inplace=True)

In [30]:
# Comprueba el resultado - asegúrate de que esté arreglado
data[data.dob_years == 0].count()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
days_employed_group    0
years_employed         0
dtype: int64

Pasamos a la columna `family_status`. Verificamos que no resalte ningun dato problematico

In [31]:
# Veamos los valores de la columna
data.family_status.value_counts().apply(porcentaje)

married              57.525822
civil partnership    19.431925
unmarried            13.061033
divorced              5.535211
widow / widower       4.446009
Name: family_status, dtype: float64

No encontramos ningun dato problematico en la columna `family_status`por lo que pasaremos a la siguiente columna

Pasamos a la columna `gender`. Verificamos que no resalte ningun dato problematico

In [32]:
# Veamos los valores en la columna
data.gender.value_counts().apply(porcentaje)

F      66.107981
M      33.887324
XNA     0.004695
Name: gender, dtype: float64

Al checar la distribución de la columna, notamos que hay valores sin indentificar. Al ser una proporción infima de los datos se decidio borrarlos para que no se genere ningun tipo de analisis en los datos.

In [33]:
# Aborda los valores problemáticos, si existen
data.drop(data[data['gender'] == 'XNA'].index, inplace=True)

In [34]:
# Comprueba el resultado - asegúrate de que esté arreglado
data.gender.value_counts().apply(porcentaje)

F    66.111085
M    33.888915
Name: gender, dtype: float64

Pasamos a la columna `income_type`. Verificamos que no resalte ningun dato problematico

In [35]:
# Veamos los valores en la columna
data.income_type.value_counts().apply(porcentaje)

employee                       51.626837
business                       23.630217
retiree                        17.921029
civil servant                   6.793746
unemployed                      0.009390
entrepreneur                    0.009390
student                         0.004695
paternity / maternity leave     0.004695
Name: income_type, dtype: float64

No encontramos ningun dato problematico en la columna `income_type`.

Finalmente, veremos si hay duplicados en nuestros datos.

In [36]:
# Comprobar los duplicados
data.duplicated().sum()

71

Se encontraron 71 datos duplicados, por lo que pasaremos a eliminarlos del dataset y asi no tener información repetida.

In [37]:
# Aborda los duplicados, si existen
data = data.drop_duplicates().reset_index(drop=True)

In [38]:
# Última comprobación para ver si tenemos duplicados
data.duplicated().sum()

0

In [39]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
data.shape

(21228, 14)

Al haber realizado todas las comprobaciones y transformaciones, tenemos un dataset con 21,228 rows y 14 columnas.
Recapitulando, se tenian 21,525 rows y 12 columnas en un inicio y se hicieron los siguientes cambios:
1. Se cambio a minisculas los datos de la columna `education` logrando asi unificar las categorias de la columna
2. Se eliminaron datos problematicos de la columna `children`
3. Para la columna `days_employed`, se crearon 2 columnas; la primera para clasificar el tiempo trabajado entre positivo y negativo y la segunda para pasar los dias de experiencia a años, una vez teniendo esta diferenciación se cambio a positivo los datos negativos y se ajustaron los datos que sobrepasaban la edad de la persona
4. Se elimaron datos problematicos de la columna `dob_years`
5. Se elimaron datos problematicos de la columna `gender`
6. Se elimaron datos duplicados de dataset

# Trabajar con valores ausentes

Finalmente, trataremos los datos ausentes presentes en las columnas `days_employed` y `total_income`.

Se habia comentado de eliminar los datos faltantes pero reflexionando sobre la cantidad en porcentaje de datos faltantes (10% en cada columna) y la naturaleza aleatoria de estos se decidio mejor restaurar estos datos con la media o mediana (se tiene que hacer un analisis para ver la mejor opcion).

Para ver cual es la mejor opción vamos a crear diccionarios para nuestras dos columnas con los calculos que necesitamos.

In [40]:
# Encuentra los diccionarios
dicc_days_employed = {'days_employed': ['mean', 'median']}
dicc_total_income = {'total_income': ['mean', 'median']}

### Restaurar valores ausentes en `total_income`

Empecemos con `total_income`. Podemos crear una columna de categoria de edad y de categoria de experiencia con el fin de tener un analisis más acotado para `total_income`

In [41]:
# Vamos a escribir una función que calcule la categoría de edad
def age_group(year):
    if year < 30:
        return 'Menor a 30'
    if year <= 60:
        return '30-60'
    if year > 60:
        return '60+'

In [42]:
# Prueba si la función funciona bien
print(age_group(18))
print(age_group(32))
print(age_group(20))
print(age_group(61))

Menor a 30
30-60
Menor a 30
60+


In [43]:
# Crear una nueva columna basada en la función
data['dob_years_group'] = data['dob_years'].apply(age_group)

In [44]:
# Comprobar cómo son los valores en la nueva columna
data['dob_years_group'].value_counts()

30-60         15947
Menor a 30     3163
60+            2118
Name: dob_years_group, dtype: int64

Generalmente los ingresos suelen depender del nivel de educación, de la experiencia de la persona y la edad de esta. Sin embargo, como tenemos valores nulos en la columna `days_employed` no podemos ocuparla. Vamos a ocupar entonces las columnas `education` y `dob_years_group`.

Antes tenemos que verificar si hay datos atipicos ya que estos pueden afectar nuestro analisis final; veamos la distribucion de estos para ver si tenemos algun dato atipico que pueda afectar.

Primero generemos datos sin estas ausencias para poder ver la distribución correctamente

In [45]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
data_sin_ausentes = data.drop(data[data['total_income'].isna()].index)
data_sin_ausentes = data_sin_ausentes.drop(data_sin_ausentes[data_sin_ausentes['days_employed'].isna()].index)

In [46]:
# Examina los valores medios de los ingresos en función de los factores que identificaste
data_grouped = data_sin_ausentes.groupby(['education', 'dob_years_group']).agg(dicc_total_income)
data_grouped

total_income            
                                             mean      median
education           dob_years_group                          
bachelor's degree   30-60            27360.289853  24029.9985
                    60+              27585.154652  22918.9550
                    Menor a 30       25824.070000  25824.0700
graduate degree     60+              28334.215000  28334.2150
primary education   30-60            18202.613951  17944.1910
                    60+              17528.709038  15695.2610
                    Menor a 30       27742.701000  27742.7010
secondary education 30-60            21831.654031  18959.6260
                    60+              20330.848122  17878.8920
                    Menor a 30       19773.760000  17931.5490
some college        30-60            18218.044263  17872.2450
                    60+              26775.310312  23289.1975
                    Menor a 30       31926.400000  31926.4000

Viendo los resultados de la tabla, vemos claramente como los valores varian relativamente mucho entre la media y la mediana. Esto nos hace sentido ya que en ingresos no hay topes como tal y una persona puede ganar muchisimo más que otra aunque tengan las misma educación y edad.

Tomando en cuenta estos descubrimientos, se decidio tomar la mediana para rellenar los valores faltantes en la columna `total_income`.

In [47]:
#  Escribe una función que usaremos para completar los valores ausentes
def llenado_income(row):
    total_income = row['total_income']
    education = row['education']
    dob_years_group = row['dob_years_group']
    if np.isnan(total_income):
        mediana = data_grouped.xs(education).xs(dob_years_group)['total_income']['median']
        return mediana
    else:
        return total_income

In [48]:
# Compruebamos si funciona
## Creamos una dataset de pruebas
data_prueba = {'education':["bachelor's degree", 'graduate degree', 'primary education', 'secondary education', 'some college'], 
               'dob_years_group':['30-60', '60+', 'Menor a 30', 'Menor a 30', '30-60'], 
               'total_income':[float("nan"), float("nan"), float("nan"), 3198, float("nan")]}
df_prueba = pd.DataFrame(data_prueba)
df_prueba

,education,dob_years_group,total_income
0,bachelor's degree,30-60,NaN
1,graduate degree,60+,NaN
2,primary education,Menor a 30,NaN
3,secondary education,Menor a 30,3198.0
4,some college,30-60,NaN


In [49]:
## Aplicamos la función al dataset de prueba y verificamos
df_prueba['total_income'] = df_prueba.apply(llenado_income, axis=1)
df_prueba

,education,dob_years_group,total_income
0,bachelor's degree,30-60,24029.9985
1,graduate degree,60+,28334.2150
2,primary education,Menor a 30,27742.7010
3,secondary education,Menor a 30,3198.0000
4,some college,30-60,17872.2450


In [50]:
# Aplícalo a cada fila
data['total_income'] = data.apply(llenado_income, axis=1)

In [51]:
# Comprueba si tenemos algún error
data.isna().sum()

children                   0
days_employed          17727
dob_years                  0
education                  0
education_id               0
family_status              0
family_status_id           0
gender                     0
income_type                0
debt                       0
total_income               0
purpose                    0
days_employed_group     2081
years_employed         17727
dob_years_group            0
dtype: int64

Como podemos checar, hemos restaurado exitosamente los valores de la columna `total_income` y ya no tenemos datos nulos en ella.
Prosigamos con `days_employed`

###  Restaurar valores en `days_employed`

Como en la columna anterior que restauramos, vamos a checar que columnas podrian darnos los datos para rellenar `days_employed`.

Consideramos que la edad de la persona es un buen parametro para ver la distribución de los años trabajados. Esto debido a que a más edad, mayor tiempo de trabajo (en la mayorias de los casos).

In [52]:
# Distribución de las medias y medianas de `days_employed` en función de los parámetros identificados
data_grouped = data_sin_ausentes.groupby(['dob_years_group']).agg(dicc_days_employed)
data_grouped

days_employed             
                         mean       median
dob_years_group                           
30-60             3521.233664  3629.802932
60+               3649.364922  3658.431689
Menor a 30         720.380520    38.967051

Viendo los resultados de la tabla, podemos notar que tenemos diferencias significativas entre la media y la mediana (principalmente en la categoria de menor a 30 años), tomando en cuenta el contexto se considera que en este caso la media nos puede dar valores más reales.

In [53]:
#  Escribe una función que usaremos para completar los valores ausentes
def llenado_employed(row):
    days_employed = row['days_employed']
    dob_years_group = row['dob_years_group']
    if np.isnan(days_employed):
        media = data_grouped.xs(dob_years_group)['days_employed']['mean']
        return media
    else:
        return days_employed

In [54]:
# Comprueba que la función funciona
## Creamos una dataset de pruebas
data_prueba = {'dob_years_group':['30-60', '60+', 'Menor a 30', 'Menor a 30', '60+'],
               'days_employed':[float("nan"), 5000, float("nan"), 1000, float("nan")]}
df_prueba = pd.DataFrame(data_prueba)
df_prueba

,dob_years_group,days_employed
0,30-60,NaN
1,60+,5000.0
2,Menor a 30,NaN
3,Menor a 30,1000.0
4,60+,NaN


In [55]:
## Aplicamos la función al dataset de prueba y verificamos
df_prueba['days_employed'] = df_prueba.apply(llenado_employed, axis=1)
df_prueba

,dob_years_group,days_employed
0,30-60,3521.233664
1,60+,5000.000000
2,Menor a 30,720.380520
3,Menor a 30,1000.000000
4,60+,3649.364922


In [56]:
# Aplícalo a cada fila
data['days_employed'] = data.apply(llenado_employed, axis=1)

In [57]:
# Comprueba si tenemos algún error
data.isna().sum()

children                   0
days_employed              0
dob_years                  0
education                  0
education_id               0
family_status              0
family_status_id           0
gender                     0
income_type                0
debt                       0
total_income               0
purpose                    0
days_employed_group     2081
years_employed         17727
dob_years_group            0
dtype: int64

Las columnas `days_employed_group` y `years_employed` fueron ocupadas para el preproceso de los datos, por lo que podemos volver a calcularlas o eliminarlas. `years_employed` nos podrian ayudar en el análisis final por lo que se volvera a calcular mientras que la columna `days_employed_group` sera eliminada

In [58]:
# Se recalcula years_employed con la data sin ausentes
data['years_employed'] = data.days_employed.apply(get_years_experience)

In [59]:
# Se elimina days_employed_group del dataset
data.drop(['days_employed_group'], axis=1, inplace=True)

In [60]:
# Verificamos el dataset final
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employed      0
dob_years_group     0
dtype: int64

Como podemos checar, hemos restaurado exitosamente los valores de la columna `days_employed`, re-calculado la columna `years_employed` por lo que ya no tenemos datos ausentes en esas columnas. Tambien eliminamos exitosamente la columna `days_employed_group`.

Una vez realizados estos pasos, nuestros datos estan libres de ausentos y listos para el análisis final.

## Clasificación de datos
Para responder las hipotesis que planteamos al inicio de nuestro analisis, vamos a tener que clasificar algunas columnas, en particular:

1. `purpose` (texto) para tener una visión más condensada de los propositos del prestamo y ver si podemos agrupar lo que la gente comento en unos propositos más generales.

2. `total_income` (numerico) para condensar en grupos más acotados los ingresos y poder generar el analisis.

Empezemos con la columna `purpose`

In [61]:
# Muestra los valores de los datos seleccionados para la clasificación
data.purpose.head(10)

0                  purchase of the house
1                           car purchase
2                  purchase of the house
3                supplementary education
4                      to have a wedding
5                  purchase of the house
6                   housing transactions
7                              education
8                       having a wedding
9    purchase of the house for my family
Name: purpose, dtype: object

In [62]:
# Comprobar los valores únicos
data.purpose.value_counts()

wedding ceremony                            785
having a wedding                            759
to have a wedding                           755
real estate transactions                    669
buy commercial real estate                  655
buying property for renting out             647
transactions with commercial real estate    643
housing transactions                        641
purchase of the house for my family         636
housing                                     635
purchase of the house                       634
property                                    627
construction of own property                626
transactions with my real estate            623
building a property                         619
purchase of my own house                    618
building a real estate                      617
buy real estate                             611
housing renovation                          602
buy residential real estate                 599
buying my own car                       

Al leer los diferentes motivos que las personas comentaron para pedir el prestamo, notamos que hay 4 categorias en las que podemos englobar nuestros datos:

1. Realizar su boda
2. Renovar, construir o comprar un patrimonio
3. Realizar estudios superiores
4. Comprar un automovil

Teniendo identificados estas categorias, haremos el agrupamiento en una columna nueva llamada `purpose_group`

In [63]:
# Escribamos una función para clasificar los datos en función de temas comunes
def purpose_group(purpose):
    if "wedding" in purpose:
        return "Wedding";
    if ("real estate" in purpose) or ("property" in purpose) or ("hous" in purpose):
        return "House / Real State";
    if "car" in purpose:
        return "Car";
    if ("educat" in purpose) or ("university" in purpose):
        return "Education"
    else:
        return "0"

In [64]:
# Crea una columna con las categorías y cuenta los valores en ellas
data['purpose_group'] = data.purpose.apply(purpose_group)
data.purpose_group.value_counts()

House / Real State    10702
Car                    4258
Education              3969
Wedding                2299
Name: purpose_group, dtype: int64

A continuación pasaremos con el caso de la columna `total_income`. Para clasificar estos datos, debemos generar una estadistica resumida de ellos y ver en que clusters podemos hacer la agrupación

In [65]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación
data.total_income

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21223    35966.698
21224    24959.969
21225    14347.610
21226    39054.888
21227    13127.587
Name: total_income, Length: 21228, dtype: float64

In [66]:
# Obtener estadísticas resumidas para la columna
data.total_income.describe()

count     21228.000000
mean      26158.861145
std       15829.883042
min        3306.762000
25%       17199.970250
50%       22547.142500
75%       31383.030500
max      362496.645000
Name: total_income, dtype: float64

Una vez viendo la estadistica, podemos notar que el dato minimo de la columna es 3,306 y el máximo es 362,496, sin embargo los datos estan distribuidos principalemente entre los 10,000 y los 30,000  (vemos que el 3er cuartil llega a 31,383) por lo que agruparemos los datos en estas clasificaciones:

1. Menos a 10,000
2. Entre 10,000 y 20,000
3. Entre 20,000 y 30,000
4. Mayor a 30,000

In [67]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos
def total_income_group(total_income):
    if total_income < 10000:
        return "Menor a 10,000"
    if total_income < 20000:
        return "Entre 10,000 y 20,000"
    if total_income < 30000:
        return "Entre 20,000 y 30,000"
    if total_income > 30000:
        return "Mayor a 30,000"

In [68]:
# Crear una columna con categorías
data['total_income_group'] = data.total_income.apply(total_income_group)

In [69]:
# Contar los valores de cada categoría para ver la distribución
data.total_income_group.value_counts()

Entre 10,000 y 20,000    7900
Entre 20,000 y 30,000    6529
Mayor a 30,000           5883
Menor a 10,000            916
Name: total_income_group, dtype: int64

Una vez terminadas nuestras clasificaciones, estamos listos para pasar a la parte final.

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [70]:
# Comprueba los datos sobre los hijos y los pagos puntuales
hijos_pagos = data.groupby('children').agg({'debt': ['size', 'sum']})

# Calcular la tasa de incumplimiento en función del número de hijos
hijos_pagos['incumplimiento'] = (hijos_pagos['debt']['sum'] / hijos_pagos['debt']['size']) * 100
hijos_pagos

debt       incumplimiento
           size   sum               
children                            
0         14019  1058       7.546901
1          4792   441       9.202838
2          2039   194       9.514468
3           328    27       8.231707
4            41     4       9.756098
5             9     0       0.000000

**Conclusión**

Como podemos ver en la tabla, las categorias que menos incumplimiento tienen son las personas con ningun hijo con 7.54% y sorprendentemente las personas con 5 hijos (0%). Las personas con 1 o hasta 4 hijos presentan mayores tasas de incumplimiento. No podemos asegurar una correlación como tal viendo los datos ya que uno podria pensar que a más hijos, mayor seria la imposibilidad de pagar un prestamo sin embargo las personas con 3 hijos presentan una menor tasa de incumplimiento que las personas con 1 o 2 hijos.

Con toda esta información, consideramos que el número de hijos podria ser considerado como una métrica secundaria en el impacto de si una persona puede o no caer en el incumplimiento de un pago.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [71]:
# Comprueba los datos del estado familiar y los pagos a tiempo
estadofamiliar_pagos = data.groupby('family_status').agg({'debt': ['size', 'sum']})

# Calcular la tasa de incumplimiento basada en el estado familiar
estadofamiliar_pagos['incumplimiento'] = (estadofamiliar_pagos['debt']['sum'] / estadofamiliar_pagos['debt']['size']) * 100
estadofamiliar_pagos

debt      incumplimiento
                    size  sum               
family_status                               
civil partnership   4112  383       9.314202
divorced            1179   84       7.124682
married            12212  923       7.558140
unmarried           2779  272       9.787693
widow / widower      946   62       6.553911

**Conclusión**

Al ver la tabla podemos notar que las categorias con mayor tasa de incumplimiento en sus pagos son en primer lugar las personas solteras y en segundo lugar las personas en una union civil. Del otro lado tenemos que las personas viudas son las personas con menor tasa de incumplimiento.

Asegurar una correlación entre el estado civil de una persona con la posibilidad de que incumpla con sus pagos podria ser dificil en este caso, sin embargo algo que si podemos concluir es que las personas sin ningun compromiso civil actual o pasado son las que más tienden a incumplir en sus creditos y podria considerarse un factor a tomar en cuenta como metrica secundaria.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [72]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
totalincome_pagos = data.groupby('total_income_group').agg({'debt': ['size', 'sum']})

# Calcular la tasa de incumplimiento basada en el nivel de ingresos
totalincome_pagos['incumplimiento'] = (totalincome_pagos['debt']['sum'] / totalincome_pagos['debt']['size']) * 100
totalincome_pagos

debt      incumplimiento
                       size  sum               
total_income_group                             
Entre 10,000 y 20,000  7900  688       8.708861
Entre 20,000 y 30,000  6529  543       8.316741
Mayor a 30,000         5883  435       7.394187
Menor a 10,000          916   58       6.331878

**Conclusión**

Viendo la tabla con sus resultados podemos notar que las personas con un ingreso bajo (menor a 10,000) y alto (mayor a 30,000) son las que menos tasa de incumplimiento presentan mientras que las personas con un rango de ingreso medio (10,000-20,000) y medio-alto(20,000-30,000) son las que tienen una tasa más alta.

Consideramos esta metrica con un impacto significativo en el posible incumplimiento de un prestamos ya que refleja la posibilidad de la persona de pagar este mismo.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [73]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos
purpose_pagos = data.groupby('purpose_group').agg({'debt': ['size', 'sum']})
purpose_pagos['incumplimiento'] = (purpose_pagos['debt']['sum'] / purpose_pagos['debt']['size']) * 100
purpose_pagos

debt      incumplimiento
                     size  sum               
purpose_group                                
Car                  4258  397       9.323626
Education            3969  369       9.297052
House / Real State  10702  777       7.260325
Wedding              2299  181       7.872988

**Conclusión**

Viendo la tabla y sus resultados, podemos ver que las categorias con mayor tasa de incumplimiento son las personas que su proposito del prestamo es comprar un carro o realizar estudios superiores. 
Mientras que la categoria con menor tasa de incumplimiento es para comprar/construir o renovar su patrimonio.

Consideramos esta métrica con un impacto significativo en el posible incumplimiento de un prestamo ya que esta nos puede reflejar si el motivo de este prestamo puede ser de bajo o alto riesgo.

# Conclusión general 

En general, los datos presentaron muchos datos problemáticos que se tuvieron que analizar y corregir, sin mencionar los datos ausentes que se tuvieron en las columnas `days_employed` y `total_income`. Estos fueron exitosamente rellandos clasificando la media y mediana por diferentes parametros. Finalmente clasificamos las columnas de `total_income` y `purpose` con el fin de poder generar un analisis más específico.

[Enumera tus conclusiones con respecto a las preguntas planteadas aquí también.]
Con respecto a nuestras hipotesis, llegamos a estas conclusiones finales:

- ¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?

Hay mucha disparidad en los resultados sin embargo pudimos concluir que las personas sin hijos tienen menor tasa de incumplimiento y consideramos que esta métrica deberia ser secundaria al decidir si una persona recibe o no un préstamo.

- ¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?

Pudimos comcluir que las personas sin ningun tipo de compromiso civil tienen un tasa de incumplimiento más alta que las demás sin embargo la conexión no las consideramos muy significativa por lo que esta métrica debería ser secundaria al decidir si una persona recibe o no un préstamo.

- ¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?

Aqui pudimos concluir que las personas con un ingreso medio (10,000-20,000) y medio-alto (20,000-30,000) son las que más alta tasa de incumplimiento presentan y consideramos que hay una conexión significativa entre el nivel de ingresos y el pago a tiempo ya que esta métrica nos refleja la capacidad de pago de cada persona y debe ser considerada como métrica primaria al decidir si una persona recibe o no un préstamo.

- ¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?

Al realizar el análisis concluimos que es un métrica primaria ya que vemos una relación entre motivos de alto riesgo con tasas de incumplimiento más altas (compra de autos, etc) mientras que los motivos más estables presentan menor tasa de incumplimiento (compra de patrimonio, etc).